In [59]:
import pandas as pd
import numpy as np

In [3]:
expeditions = pd.read_csv('./data/expeditions.csv')
members = pd.read_csv('./data/members.csv')
peaks = pd.read_csv('./data/peaks.csv')

### Question 1

In [42]:
# Get proportion of peaks unclimbed
unclimbed_one = (peaks[peaks['climbing_status'] == 'Unclimbed'].value_counts('climbing_status')/len(peaks))
unclimbed_one # around 27% are still unclimbed


climbing_status
Unclimbed    0.271368
dtype: float64

In [43]:
#OR

unclimbed_two = (peaks['climbing_status'].value_counts()['Unclimbed']/len(peaks))
unclimbed_two

0.27136752136752135

### TRY NORMALIZE

In [33]:
# Get Avg. Height of climbed & unclimbed

avg_height = peaks.groupby('climbing_status')['height_metres'].mean()
avg_height # 6706.3 for Climbed, 6523.3 for Unclimbed

climbing_status
Climbed      6706.284457
Unclimbed    6523.330709
Name: height_metres, dtype: float64

### Question 2

In [66]:
members_citizenship = members[members['citizenship'] == 'Nepal']
members_citizenship # get Sherpas

,expedition_id,member_id,peak_id,peak_name,year,season,sex,age,citizenship,expedition_role,...,highpoint_metres,success,solo,oxygen_used,died,death_cause,death_height_metres,injured,injury_type,injury_height_metres
24,AMAD79101,AMAD79101-18,AMAD,Ama Dablam,1979,Spring,M,23.0,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
48,AMAD79301,AMAD79301-26,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
49,AMAD79301,AMAD79301-24,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
50,AMAD79301,AMAD79301-25,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
82,AMAD81101,AMAD81101-11,AMAD,Ama Dablam,1981,Spring,M,25.0,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76506,PUMO19102,PUMO19102-01,PUMO,Pumori,2019,Spring,M,35.0,Nepal,Leader,...,7138.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76507,PUMO19102,PUMO19102-02,PUMO,Pumori,2019,Spring,M,52.0,Nepal,H-A Worker,...,7138.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76513,PUTH19101,PUTH19101-06,PUTH,Putha Hiunchuli,2019,Spring,M,NaN,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN
76514,PUTH19101,PUTH19101-07,PUTH,Putha Hiunchuli,2019,Spring,M,44.0,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN


In [56]:
members_cit_hired = members_citizenship['hired'].value_counts()[True]/len(members_citizenship)
members_cit_hired # hired percentage ~ 93.4%

0.9340563991323211

In [69]:
members_cit_age = members_citizenship[['age','citizenship']].groupby('citizenship').agg(['min', 'max', 'mean'])
members_cit_age

age                 
              min   max       mean
citizenship                       
Nepal        12.0  85.0  32.885062

### Question 3

FIX MERGES, SOMETHING IS OFF ABOUT THEM!!!!

In [333]:
merged_data = pd.merge(pd.merge(peaks, expeditions, 
         on = 'peak_id'), members[members['hired'] == False],
         on = 'expedition_id', indicator = True).drop_duplicates()

merged_data

,peak_id_x,peak_name_x,peak_alternative_name,height_metres,climbing_status,first_ascent_year,first_ascent_country,first_ascent_expedition_id,expedition_id,peak_name_y,...,success,solo,oxygen_used_y,died,death_cause,death_height_metres,injured,injury_type,injury_height_metres,_merge
0,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88101,Ama Dablam,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
1,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88101,Ama Dablam,...,False,False,False,False,NaN,NaN,True,AMS,NaN,both
2,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88101,Ama Dablam,...,False,False,False,False,NaN,NaN,True,Falling rock / ice,NaN,both
3,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88101,Ama Dablam,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
4,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88102,Ama Dablam,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60777,JANE,Jannu East,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE18301,Jannu East,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
60778,JANE,Jannu East,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE18301,Jannu East,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
60779,JANE,Jannu East,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE19101,Jannu East,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both
60780,JANE,Jannu East,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE19101,Jannu East,...,False,False,False,False,NaN,NaN,False,NaN,NaN,both


In [316]:
first_female = merged_data[(merged_data['sex'] == 'F') & (merged_data['success'] == True)].nsmallest(1, 'year', keep = 'all')[['peak_name', 'height_metres', 'year']]

first_female


,peak_name,height_metres,year
26609,Tengkoma,6215,1949
26820,Dzanye,6581,1949


In [327]:
merged_data[['success', 'oxygen_used', 'died', 'sex']].groupby('sex').apply(lambda x: x.sum()/len(x))

,success,oxygen_used,died
sex,,,
F,0.676634,0.389334,0.008323
M,0.628180,0.297567,0.016642


In [326]:
#run checks

members_test = members[['success', 'oxygen_used', 'died', 'sex', 'hired']]
members_test = members_test[(members_test['hired'] == False) & (members_test['sex'] == 'F')]
members_test

,success,oxygen_used,died,sex,hired
25,False,False,False,F,False
28,False,False,False,F,False
37,False,False,False,F,False
46,False,False,False,F,False
93,False,False,False,F,False
...,...,...,...,...,...
76455,True,True,False,F,False
76458,True,False,False,F,False
76464,True,True,False,F,False
76467,True,True,False,F,False


In [296]:
members_test[['success', 'died']].apply(lambda x: x.sum()/len(x))

success    0.326583
died       0.006988
dtype: float64

In [297]:
sum(members_test['success'])/len(members_test)


0.32658300057045064

In [310]:
test1 = merged_data[['success', 'oxygen_used', 'died', 'sex']]
test1 = merged_data[merged_data['sex'] == 'F']
test1

,peak_id,peak_name,peak_alternative_name,height_metres,climbing_status,first_ascent_year,first_ascent_country,first_ascent_expedition_id,expedition_id,year,...,hired,highpoint_metres_y,success,solo,died,death_cause,death_height_metres,injured,injury_type,injury_height_metres
20,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88402,1988,...,False,NaN,False,False,False,NaN,NaN,False,NaN,NaN
24,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88402,1988,...,False,6814.0,True,False,False,NaN,NaN,False,NaN,NaN
27,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD88402,1988,...,False,NaN,False,False,False,NaN,NaN,False,NaN,NaN
29,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD83301,1983,...,False,6814.0,True,False,False,NaN,NaN,False,NaN,NaN
32,AMAD,Ama Dablam,Amai Dablang,6814,Climbed,1961.0,"New Zealand, USA, UK",AMAD61101,AMAD83301,1983,...,False,NaN,False,False,False,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49912,TARS,Tarke Kang Shar,Tare Kang,7069,Climbed,1981.0,"Italy, Nepal",TARS81301,TARS90301,1990,...,False,NaN,False,False,False,NaN,NaN,False,NaN,NaN
49933,KORL,Korlang Pari Tippa,Korlang Pari Tippa North,5738,Unclimbed,NaN,NaN,NaN,KORL18301,2018,...,False,5574.0,False,False,False,NaN,NaN,False,NaN,NaN
49941,JANE,Jannu East,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE04301,2004,...,False,NaN,False,False,False,NaN,NaN,False,NaN,NaN
49942,JANE,Jannu East,Khumbhakarna East,7460,Unclimbed,NaN,NaN,NaN,JANE04301,2004,...,False,NaN,False,False,False,NaN,NaN,False,NaN,NaN


In [315]:
a = peaks.columns.intersection(expeditions.columns)
print('Common Columns',a)

Common Columns Index(['peak_id', 'peak_name'], dtype='object')


In [330]:
members

,expedition_id,member_id,peak_id,peak_name,year,season,sex,age,citizenship,expedition_role,...,highpoint_metres,success,solo,oxygen_used,died,death_cause,death_height_metres,injured,injury_type,injury_height_metres
0,AMAD78301,AMAD78301-01,AMAD,Ama Dablam,1978,Autumn,M,40.0,France,Leader,...,NaN,False,False,False,False,NaN,NaN,False,NaN,NaN
1,AMAD78301,AMAD78301-02,AMAD,Ama Dablam,1978,Autumn,M,41.0,France,Deputy Leader,...,6000.0,False,False,False,False,NaN,NaN,False,NaN,NaN
2,AMAD78301,AMAD78301-03,AMAD,Ama Dablam,1978,Autumn,M,27.0,France,Climber,...,NaN,False,False,False,False,NaN,NaN,False,NaN,NaN
3,AMAD78301,AMAD78301-04,AMAD,Ama Dablam,1978,Autumn,M,40.0,France,Exp Doctor,...,6000.0,False,False,False,False,NaN,NaN,False,NaN,NaN
4,AMAD78301,AMAD78301-05,AMAD,Ama Dablam,1978,Autumn,M,34.0,France,Climber,...,NaN,False,False,False,False,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76514,PUTH19101,PUTH19101-07,PUTH,Putha Hiunchuli,2019,Spring,M,44.0,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN
76515,RATC19101,RATC19101-01,RATC,Ratna Chuli,2019,Spring,M,55.0,Romania,Leader,...,6600.0,False,False,False,False,NaN,NaN,False,NaN,NaN
76516,SANK19101,SANK19101-01,SANK,Sano Kailash,2019,Spring,M,56.0,Austria,Leader,...,6452.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76517,SANK19101,SANK19101-02,SANK,Sano Kailash,2019,Spring,F,57.0,Austria,Climber,...,5700.0,False,False,False,False,NaN,NaN,False,NaN,NaN
